http://scikit-learn.org/stable/modules/naive_bayes.html

http://sklearn.lzjqsdd.com/modules/naive_bayes.html

Iris数据集的输出空间为(Setosa, Versicolour, Virginica)，也就是训练的输出只可能是这三个；输入则是一个简单的四维向量，包含了Sepal length、Sepal width、Petal length和Petal width 四个特征。

# GaussianNB

In [41]:
from sklearn import datasets
iris = datasets.load_iris()
from sklearn.naive_bayes import GaussianNB
iris

{'DESCR': 'Iris Plants Database\n====================\n\nNotes\n-----\nData Set Characteristics:\n    :Number of Instances: 150 (50 in each of three classes)\n    :Number of Attributes: 4 numeric, predictive attributes and the class\n    :Attribute Information:\n        - sepal length in cm\n        - sepal width in cm\n        - petal length in cm\n        - petal width in cm\n        - class:\n                - Iris-Setosa\n                - Iris-Versicolour\n                - Iris-Virginica\n    :Summary Statistics:\n\n    ============== ==== ==== ======= ===== ====================\n                    Min  Max   Mean    SD   Class Correlation\n    ============== ==== ==== ======= ===== ====================\n    sepal length:   4.3  7.9   5.84   0.83    0.7826\n    sepal width:    2.0  4.4   3.05   0.43   -0.4194\n    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)\n    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)\n    ============== ==== ==== ======= ===== ==

In [42]:
gnb = GaussianNB()
y_pred = gnb.fit(iris.data,iris.target).predict(iris.data)
print("Number of mislabeled points out of a total %d points : %d"%(iris.data.shape[0],(iris.target!=y_pred).sum()))      

Number of mislabeled points out of a total 150 points : 6


参考网址：https://zhuanlan.zhihu.com/p/26055646

# MultinomialNB多项式模型

http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB

In [44]:
import numpy as np
X = np.random.randint(5,size=(6,100))#生成指定范围内的整数，比如此例中小于5的
y = np.array([1,2,3,4,5,6])
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X,y)
print(clf.predict(X[2:3]))

[3]


# BernoulliNB伯努力模型

简化：将重复的词语都视为其只出现1次

In [45]:
import numpy as np
X = np.random.randint(2,size=(6,100))
Y = np.array([1,2,3,4,4,5])
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(X,Y)
print(clf.predict(X[2:3]))

[3]


# 朴素贝叶斯分类器的Python实现

## 定义特征提取器

In [58]:
def gender_features(word):
    return {'last_letter':word[-1]}
gender_features("Shrek")

{'last_letter': 'k'}

## 准备数据

In [63]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [64]:
from nltk.corpus import names
import random
names = ([(name, 'male') for name in names.words('male.txt')] + [(name, 'female') for name in names.words('female.txt')])
random.shuffle(names)

## 朴素贝叶斯分类器

In [66]:
featuresets = [(gender_features(n),g) for (n,g) in names]
train_set,test_set = featuresets[500:],featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
classifier.classify(gender_features("Neo"))

'male'

## 评估分类器

In [67]:
print (nltk.classify.accuracy(classifier, test_set))

0.728


检查分类器，确定哪些特征对于区分名字的性别是最有效的:

In [68]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'a'            female : male   =     35.9 : 1.0
             last_letter = 'k'              male : female =     31.1 : 1.0
             last_letter = 'f'              male : female =     15.2 : 1.0
             last_letter = 'p'              male : female =     12.5 : 1.0
             last_letter = 'v'              male : female =     10.5 : 1.0


## 优化

在处理大型语料库时，构建包含所有实例特征的单独链表会占用大量内存。可以使用函数 nltk.classify.apply_features， 类似生成器原理。

In [69]:
from nltk.classify import apply_features
train_set = apply_features(gender_features,featuresets[500:])
test_set = apply_features(gender_features,featuresets[:500])
type(train_set)

nltk.collections.LazyMap

## 结果评估与改进

将数据分为训练集、开发测试集和测试集。

训练集用于训练模型，开发测试集用于执行错误分析，测试集用于系统的最终评估。

In [70]:
train_names = names[1500:]
devtest_names = names[500:1500]
test_names = names[:500]
train_set = [(gender_features(n), g) for (n, g) in train_names]
devtest_set = [(gender_features(n), g) for (n, g) in devtest_names]
test_set = [(gender_features(n), g) for (n, g) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print (nltk.classify.accuracy(classifier, devtest_set))

0.764


In [72]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append((tag, guess, name))
for (tag, guess, name) in sorted(errors):
    print ('correct=%-8s guess=%-8s name=%-30s' % (tag, guess, name))

correct=female   guess=male     name=Adelind                       
correct=female   guess=male     name=Alleen                        
correct=female   guess=male     name=Allsun                        
correct=female   guess=male     name=Alyson                        
correct=female   guess=male     name=Anabel                        
correct=female   guess=male     name=Angil                         
correct=female   guess=male     name=Annabal                       
correct=female   guess=male     name=Ardelis                       
correct=female   guess=male     name=Beatriz                       
correct=female   guess=male     name=Beau                          
correct=female   guess=male     name=Beilul                        
correct=female   guess=male     name=Bess                          
correct=female   guess=male     name=Bette-Ann                     
correct=female   guess=male     name=Bidget                        
correct=female   guess=male     name=Bill       

根据上面的错误列表改进特征提取器，比如发现yn结尾的名字大多以女性为主，但以n结尾的名字往往是男性，因此调整特征提取器使其包含两个字母后缀的特征：

In [ ]:
def gender_features(word):
    return {'suffix1':word[-1:],
            'suffix2':word[-2:]}